In [10]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

In [2]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='my-test-experiment', tags={}>]

In [4]:
client.create_experiment(name = "my-test-experiment")

'2'

In [8]:
runs = client.search_runs(
    experiment_ids = "1", 
    filter_string = "metrics.rmse < 6.8",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5,
    order_by = ["metrics.rmse ASC"]
)

In [9]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: a350890d76a74424b5b9513a627faae5, rmse: 6.3158
run id: 63541f49c408476b9008188df3349eeb, rmse: 6.3211
run id: e73384e71602492e8bc9e97e49cccfbb, rmse: 6.3233
run id: a9e8d255be124df5981871c71bf4969c, rmse: 6.3340
run id: d0c1776c09894b7d81afb131a3b4457f, rmse: 6.3355


In [11]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [12]:
model_uri = "runs:/d51d2beaad2046e798b50d13b9967c49/model"
mlflow.register_model(model_uri = model_uri, name = "nyc-taxi-experiment")

Registered model 'nyc-taxi-experiment' already exists. Creating a new version of this model...
2022/05/29 12:22:36 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-experiment, version 2
Created version '2' of model 'nyc-taxi-experiment'.


<ModelVersion: creation_timestamp=1653837756092, current_stage='None', description=None, last_updated_timestamp=1653837756092, name='nyc-taxi-experiment', run_id='d51d2beaad2046e798b50d13b9967c49', run_link=None, source='./mlruns/1/d51d2beaad2046e798b50d13b9967c49/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [17]:
latest_versions = client.get_latest_versions(name = "nyc-taxi-experiment")

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging


In [16]:
client.transition_model_version_stage(
    name = "nyc-taxi-experiment",
    version = 2,
    stage = "Staging",
    archive_existing_versions = False
)

<ModelVersion: creation_timestamp=1653837756092, current_stage='Staging', description=None, last_updated_timestamp=1653837954223, name='nyc-taxi-experiment', run_id='d51d2beaad2046e798b50d13b9967c49', run_link=None, source='./mlruns/1/d51d2beaad2046e798b50d13b9967c49/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [18]:
client.update_model_version(
    name = "nyc-taxi-experiment",
    version = 2,
    description = f"The model version 2 was transitioned to Staging"
)

<ModelVersion: creation_timestamp=1653837756092, current_stage='Staging', description='The model version 2 was transitioned to Staging', last_updated_timestamp=1653838046927, name='nyc-taxi-experiment', run_id='d51d2beaad2046e798b50d13b9967c49', run_link=None, source='./mlruns/1/d51d2beaad2046e798b50d13b9967c49/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>